In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np 


# Q1

In [13]:
import torch.nn.functional as F
image = torch.rand(6,6)
for i in range(2):
    image = image.unsqueeze(dim=0)
print("Image shape: ",image.shape)

kernel = torch.rand(3,3)
for i in range(2):
    kernel = kernel.unsqueeze(dim=0)
print("Kernel : ",kernel.shape)
# General Parameters
output = F.conv2d(image,kernel,stride=1,padding=0)
print("Output Image: ",output.shape)
# Changing Padding Parameter
output = F.conv2d(image,kernel,stride=1,padding=1)
print("Output Image-2: ",output.shape)
# 




Image shape:  torch.Size([1, 1, 6, 6])
Kernel :  torch.Size([1, 1, 3, 3])
Output Image:  torch.Size([1, 1, 4, 4])
Output Image-2:  torch.Size([1, 1, 6, 6])


# Q2

In [19]:
import torch.nn as nn
image = torch.rand(6,6)
for i in range(2):
    image = image.unsqueeze(dim=0)
print("Image shape: ",image.shape)
conv = nn.Conv2d(1,3,kernel_size=3,padding=1,bias = False)
res1 = conv(image)
weights = conv.weight
print(weights.shape)
res2 = F.conv2d(image,weights,padding = 1, bias = None)
print(res1.shape)
print(res2.shape)

Image shape:  torch.Size([1, 1, 6, 6])
torch.Size([3, 1, 3, 3])
torch.Size([1, 3, 6, 6])
torch.Size([1, 3, 6, 6])


# Q3

In [24]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torch.optim as optimizer 
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data',train = True, transform = transform)
test_dataset = datasets.MNIST(root='./data',train = False, transform = transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class CNNMnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(64,32)
        self.output = nn.Linear(32,10)
        self.maxpool = nn.MaxPool2d((2,2),stride=2)
    def forward(self,x):
        # Convolution
        x = nn.Conv2d(1,64,kernel_size=3)(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = nn.Conv2d(64,128,kernel_size=3)(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = nn.Conv2d(128,64,kernel_size=3)(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # Classification
        x = self.fc1(x.view(64,-1))
        x = self.relu(x)

        x = self.output(x)
        x = self.softmax(x)
        return x

model = CNNMnist()
criterion = nn.CrossEntropyLoss()
opt = optimizer.Adam(model.parameters(),lr = 0.001)
epochs = 20

for epoch in range(epochs):
    for images, labels in train_loader:
        opt.zero_grad()
        print(images.shape)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()
    print("Epoch {epoch} LOSS: {loss}")





torch.Size([64, 1, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4096x1 and 64x32)